# **GNU Colab**

*In here we set an environment in which you can persistently run desktop programs.*

This notebook lets you **run desktop programs on colab machines** drawn on your local terminal through VNC.

At this moment, the notebook is preconfigured to run either **MATE** or **XFCE**. 

Other options can of course be added.

**Best run**: TBD


In [ ]:
!uptime

## Setup

### Routines

#### Notebook loader

In [ ]:
#@title Imports
import io, os, sys, types

from IPython import get_ipython
from nbformat import read
from IPython.core.interactiveshell import InteractiveShell

In [ ]:
#@title `find_notebook(fullname, path=None)`
#@markdown *find a notebook, given its fully qualified name and an optional path*

#@markdown This turns `"foo.bar"` into `"foo/bar.ipynb"`
#@markdown and tries turning `"Foo_Bar"` into `"Foo Bar"` if `Foo_Bar`
#@markdown does not exist.

def find_notebook(fullname, path=None):
    """find a notebook, given its fully qualified name and an optional path

    This turns "foo.bar" into "foo/bar.ipynb"
    and tries turning "Foo_Bar" into "Foo Bar" if Foo_Bar
    does not exist.
    """
    name = fullname.rsplit('.', 1)[-1]
    if not path:
        path = ['']
    for d in path:
        nb_path = os.path.join(d, name + ".ipynb")
        if os.path.isfile(nb_path):
            return nb_path
        # let import Notebook_Name find "Notebook Name.ipynb"
        nb_path = nb_path.replace("_", " ")
        if os.path.isfile(nb_path):
            return nb_path

In [ ]:
#@title `NotebookLoader(object)`
#@markdown Module Loader for Jupyter Notebooks

class NotebookLoader(object):
    """Module Loader for Jupyter Notebooks"""
    def __init__(self, path=None):
        self.shell = InteractiveShell.instance()
        self.path = path

    def load_module(self, fullname):
        """import a notebook as a module"""
        path = find_notebook(fullname, self.path)

        print ("importing Jupyter notebook from %s" % path)

        # load the notebook object
        with io.open(path, 'r', encoding='utf-8') as f:
            nb = read(f, 4)


        # create the module and add it to sys.modules
        # if name in sys.modules:
        #    return sys.modules[name]
        mod = types.ModuleType(fullname)
        mod.__file__ = path
        mod.__loader__ = self
        mod.__dict__['get_ipython'] = get_ipython
        sys.modules[fullname] = mod

        # extra work to ensure that magics that would affect the user_ns
        # actually affect the notebook module's ns
        save_user_ns = self.shell.user_ns
        self.shell.user_ns = mod.__dict__

        try:
          for cell in nb.cells:
            if cell.cell_type == 'code':
                # transform the input to executable Python
                code = self.shell.input_transformer_manager.transform_cell(cell.source)
                # run the code in themodule
                exec(code, mod.__dict__)
        finally:
            self.shell.user_ns = save_user_ns
        return mod

In [ ]:
#@title `NotebookFinder(object)`
#@markdown Module finder that locates Jupyter Notebooks

class NotebookFinder(object):
    """Module finder that locates Jupyter Notebooks"""
    def __init__(self):
        self.loaders = {}

    def find_module(self, fullname, path=None):
        nb_path = find_notebook(fullname, path)
        if not nb_path:
            return

        key = path
        if path:
            # lists aren't hashable
            key = os.path.sep.join(path)

        if key not in self.loaders:
            self.loaders[key] = NotebookLoader(path)
        return self.loaders[key]

In [ ]:
#@title `sys.meta_path.append(NotebookFinder())`

sys.meta_path.append(NotebookFinder())

#### Utils

In [ ]:
#@title Make so that if { vertical-output: true }

run = True #@param{type:"boolean"}

from os import listdir as ls
#@markdown If the boolean `bad_condition` happens, `a_function(error, *args)` gets executed.

#@markdown i.e:
#@markdown ```python3
#@markdown empty_root = len(ls("/")) == 0
#@markdown
#@markdown def scream(error=None):
#@markdown   print("ARGH!")
#@markdown ```
empty_root =  len(ls("/")) == 0

def scream(error=None):
  print("ARGH!")

bad_condition =  empty_root #@param {type:"raw"}
a_function = scream #@param {type:"raw"}

def make_so_that_if(bad_condition=bad_condition, 
                    a_function=a_function, 
                    *a_function_arguments):
  try:
    assert not bad_condition
    return True
  except AssertionError as e:
    a_function(e, *a_function_arguments)

if run:
  make_so_that_if()

In [ ]:
#@title Record { vertical-output: true }
run = True #@param{type:"boolean"}
variable = "a python string" #@param {type:"raw"}
env_name = "a_bash_variable_name" #@param {type:"string"}

from os import environ
def record(variable=variable, env_name=env_name): 
  if variable: 
    environ[env_name] = variable
    return True

if run:
  record()
  print("${} = {}".format(env_name, environ[env_name]))

In [ ]:
#@title Save text to text file
run = True #@param {type:"boolean"}
file = "/examplefile" #@param {type:"string"}
content = "content of the file" #@param {type: "string"}


def save_text_to_file(file=file, content=content):
  with open(file, "w") as f:
    f.write(content)

if run:
  save_text_to_file()
  
  record(file, "file")
  print(file)
  print("----------------")
  !cat $file

In [ ]:
#@title Search text in a file { vertical-output: true }

run = True #@param{type:"boolean"}
path = "/examplefile" #@param{type:"string"}
pattern = "the thing to search for" #@param {type:"string"}

def search_in_file(path=path, pattern=pattern):
  with open(path, "r") as f:
    content = f.read()
    
    if pattern in content:
      print(True)
    else:
      print(False)
if run:
  search_in_file()

In [ ]:
#@title Replace text in a file { vertical-output: true }

run = False #@param{type:"boolean"}
path = "/examplefile" #@param{type:"string"}
pattern = "the thing to replace" #@param {type:"string"}
replacement = "the replacement" #@param {type:"string"}

def replace_in_file(path=path, pattern=pattern, replacement=replacement):
  with open(path, "r") as f:
    content = f.read()
    
  content = content.replace(pattern, replacement)
 
  with open(path, "w") as f:
    f.write(content)

if run:
  replace_in_file()


In [ ]:
#@title Append text to a file

run = False #@param{type:"boolean"}
file_path = "/examplefile" #@param{type:"string"}
text = "the text to append" #@param {type:"string"}

def append_to(error=None, file_path=file_path, text=text):
  record(user, "user")
  record(text, "text")
  !runuser -l $user -c "echo $text >> $path"

if run:
  append_to()

In [ ]:
#@title Enable lots of Ubuntu repositories
run = False #@param{type:"boolean"}

def enable_many_ubuntu_repos():
  !add-apt-repository universe > /dev/null 2>&1
  !add-apt-repository multiverse > /dev/null 2>&1
  !add-apt-repository restricted > /dev/null 2>&1
  !apt update > /dev/null 2>&1

if run:
  enable_many_ubuntu_repos()

### Run

In [ ]:
#@title Questions { vertical-output: true }
#@markdown For what will you use this machine for?
usage = "Development" #@param ["Development", "Gaming", "Service deployment", "Production", ""]

#@markdown Do you want to minimize the amount of proprietary software being used?
rms = False #@param {type:"boolean"}

if not rms:
  enable_many_ubuntu_repos()

#@markdown Where do you want this machine's state to be stored?
storage = "Google Drive" #@param ["Google Drive", "I have an ssh server", "In GNU Colab cloud (experimental)", ""]

if storage == "Google Drive":
  print("Google Drive storage selected:\n  proceed to the \"Google drive integration\" section")

  #@markdown How do you want to connect to this machine?
  storage = "I have a machine on which I can setup an ssh reverse tunnel" #@param ["Google Drive", "I have a machine on which I can setup an ssh reverse tunnel", "ngrok", ""]

### Debug

#### System information

In [ ]:
#@title Are we in docker? { vertical-output: true }
run = True #@param {type:"boolean"}

def in_docker():
  """ Returns: True if running in a Docker container, else False """
  with open('/proc/1/cgroup', 'rt') as ifh:
      return 'docker' in ifh.read()

if run:
  if in_docker():
    print("We are in a docker container.")

In [ ]:
#@title Are we in a privileged docker? { vertical-output: true }
run = True #@param {type:"boolean"}

def can_load_modules():
  modules = ["br_netfilter", 
             "veth",
             "virtio_balloon",
             "aesni_intel"]
  for module in modules:
    record(module, "module")
    output = !rmmod $module
    for o in output:
      if "Operation not permitted" in o:
        print(output)
        return False
  return True

if run:
  if not can_load_modules():
    print("No, since we can't unload kernel modules")

In [ ]:
#@title Get *running* kernel version { vertical-output: true }
run = True #@param {type:"boolean"}

def get_running_kernel_version():
  from os import uname
  return uname().release

if run:
  print(get_running_kernel_version())

In [ ]:
#@title Get *installed* ubuntu kernel modules version { vertical-output: true }
run = True #@param {type:"boolean"}

def get_installed_kernel_modules_version():
  from os import listdir as ls
  modules = ls("/lib/modules")
  if len(modules):
    if len(modules) > 1:
      print("installed modules versions: {}".format(modules))
    return modules[0]

if run:
  print(get_installed_kernel_modules_version())

In [ ]:
#@title Get kernel name in Ubuntu modules' declaration { vertical-output: true }
run = True #@param {type:"boolean"}

def get_running_modules_kernel_version():
  return get_running_kernel_version()[:-1] + "-generic"

if run:
  print(get_running_modules_kernel_version())

In [ ]:
#@title List available kernel modules (`ko` files) { vertical-output: true }

run = True #@param {type:"boolean"}

def list_ko_files():
  !find /lib/modules/ | grep .ko

if run:
  list_ko_files()

## User management

*In here an user `user` is set up.*

Set up a familiar work environment.

**Best run**: 12 seconds

**Worst run**: 35 seconds

In [ ]:
#@title New **user** (default: `user`) { vertical-output: true }
#@markdown **Concise. It also enables cron` service.**
user = "user" #@param {type:"string"}
password = "testone" #@param {type:"string"}
run = True #@param {type:"boolean"}
root = True #@param {type:"boolean"}

!apt update > /dev/null 2>&1
# !apt upgrade > /dev/null 2>&1

from os import environ

def create_user(user=user, password=password, root=root):

  environ['user'] = user
  environ['password'] = password
  
  # Add user
  !useradd $user > /dev/null 2>&1
  !usermod -aG sudo user
  !echo -e "$password\n$password" | passwd root

  # Directories
  !mkdir /home/$user > /dev/null 2>&1
  !chmod -R 700 /home/$user
  !mkdir -p /home/$user/Projects > /dev/null 2>&1
  !chown -R $user:$user /home/$user
  !apt install  xdg-user-dirs > /dev/null 2>&1
  !runuser -l $user -c "xdg-user-dirs-update"

  !mkdir -p /tmp/$user
  !chown -R $user:$user /tmp/$user 
 
  # Software related
  !touch /var/log/pip.log
  !chown user:user /var/log/pip.log

  # Set root password
  if root:
    !echo -e "$password\n$password" | passwd root

  # User level libraries
  # !python3 -m pip uninstall -y google-colab

if run:
  create_user()

  # Set hostname
  hostname = !hostname
  if len(hostname[0]) < 15:
    !hostname gnu-colab-$(hostname)

In [ ]:
#@title Get amenities { vertical-output: true }
run = True #@param {type:"boolean"}
#@markdown - ### Shell
#@markdown I heard you want both `user` and root to use **ohmyzsh**, am I right? 
user = "user" #@param {type:"string"}
root = True #@param {type:"boolean"}

from os import environ
def record(variable, env_name): 
  if variable: 
    environ[env_name] = variable
    return True

def setup_shell(user=user, root=root):
  from os import environ
  environ['user'] = user

  !apt --quiet install zsh nyancat nyancat-server fortune > /dev/null 2>&1
 
  !runuser -l $user -c "$(curl -fsSL https://raw.githubusercontent.com/ohmyzsh/ohmyzsh/master/tools/install.sh) --unattended" > /dev/null 2>&1

  !usermod --shell /usr/bin/zsh $user > /dev/null 2>&1
  if root:
    !sh -c "$(curl -fsSL https://raw.githubusercontent.com/ohmyzsh/ohmyzsh/master/tools/install.sh) --unattended" > /dev/null 2>&1
    !usermod --shell /usr/bin/zsh root > /dev/null 2>&1

#@markdown - ### Text editor
#@markdown Something *reasonable*

editor = "vim" #@param {type:"string"}

def setup_text_editor(editor=editor):
  record(editor, "editor")
  !apt --quiet install -y $editor > /dev/null 2>&1

#@markdown - ### Window manager
#@markdown I mean `screen`.
!apt install screen > /dev/null 2>&1

#@markdown  - ### Monitoring 
#@markdown I/O, cron, ssh, logging, gpg.

#@markdown Install HTTP monitoring?
cockpit = False #@param {type:"boolean"}

def setup_monitoring():

  # Install cron
  !apt install cron > /dev/null 2>&1

  # Install ssh
  !apt install openssh-server autossh tor > /dev/null 2>&1

  # I/O
  !apt install sysstat > /dev/null 2>&1

  # GnuPG
  !apt install gnupg > /dev/null 2>&1
  !apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 51716619E084DAB9

  # Install rsyslog
  !apt install rsyslog > /dev/null 2>&1
  # Enable cron logging
  !sed -i '/cron\./s/^#//g' /etc/rsyslog.d/50-default.conf

  if cockpit:
    packages = ("cockpit cockpit-ws cockpit-packagekit "
                "cockpit-docker cockpit-machines cockpit-dashboard "
                "cockpit-system")
    record(packages, "packages")
    !apt install --fix-missing $packages > /dev/null  2>&1
    !rm /var/lib/dpkg/statoverride

  # Restart services
  !service cron restart
  !service rsyslog restart 
  !service ssh restart

if run:
  setup_shell()
  setup_text_editor()
  setup_monitoring()

### Debug
Have fun!

In [ ]:
#@title Start a shell { vertical-output: true }
run = False #@param {type:"boolean"}
#@markdown *Why not?*
user = "root" #@param ["user", "root"] {allow-input: true}
shell = "bash" #@param {type:"string"}


def start_shell(user=user, shell=shell):
  from os import environ
  environ['user'] = user
  environ['shell'] = shell

  if user == "root":
    !$shell
  if shell == "zsh":
    !su - $user
  
if run:
  start_shell()

In [ ]:
#@title Package management
run = False #@param {type:"boolean"}
#@markdown *Light package management actions*
action = "Install" #@param ["Install", "Show installed files"] {allow-input: true}

package = "pgpgram" #@param {type:"string"}
system = "apt" #@param ["apt", "pip", ""]

def install(package=package, system=system):
  record(package, "package") 

  if system == "apt":
    !apt --fix-broken install
    !killall apt > /dev/null 2>&1
    !rm /var/lib/dpkg/lock-frontend
    !dpkg --configure -a > /dev/null 2>&1

    !apt-get  install -o Dpkg::Options::="--force-confold" --no-install-recommends --force-yes -y $package /dev/null 2>&1
    
    !dpkg --configure -a #2>&1 
    !apt  update > /dev/null 2>&1

    !apt install $package > /dev/null 2>&1

  if system == "pip":
    !runuser -l $user -c "python3 -m pip install --user $package"

def get_package_files(package=package):
  from os import environ
  environ['package'] = package
  files = !dpkg-query -L $package
  return files

if run:
  if action == "install":
    install()
  if action == "Show installed files":
    from pprint import pprint
    pprint(get_package_files())

## Google drive integration

*In here we mount your drive in `user` home.*

**Manual intervention required**: When this will be optional you won't have to open this section to insert a code.

In [ ]:
#@title Google Drive Integration { vertical-output: true }
#@markdown Check sign-in this cell to sign-in as `user`.
#@markdown In here a shortcut to your google drive is inserted inside  `user` home. It is read-only for `user`.
user = "user" #@param {type:"string"}
run = True #@param {type:"boolean"}
debug = False #@param {type:"boolean"}
mountpoint = "/content/drive"

from os import getuid, getgid, setuid, setgid
from os import environ as env
from os.path import join as path_join
from os import listdir as ls


env['user'] = user
env['user_home'] = "/home/{}".format(user)
env['drive_mount_bin'] = path_join(env['user_home'], 'drive_mount.sh')

def drive_mount():
  !runuser -l $user -c "yes | python3 -m pip install --user google-colab"  > /dev/null 2>&1

  def mount(user=user, debug=debug):
    !cp -r /root/.config/Google /home/$user/.config/Google > /dev/null 2>&1
    !mkdir -p /home/$user/.config/Google/DriveFS/Logs
    !chown -R $user:$user /home/user/.config/Google/DriveFS/Logs
    !mkdir -p /content/drive > /dev/null 2>&1
    !chown -R $user:$user /content/.config
    !chown -R $user:user /content > /dev/null 2>&1
  mount()

  mount = """
#!/bin/sh

from os import environ as env
from os.path import join as path_join
from google.colab import drive

env['CLOUDSDK_CONFIG']  = '/content/.config'

def drive_mount(mountpoint=path_join('/content', 'drive'), user="{}"):
  try:
    drive.mount(mountpoint)
  except Exception as e:
    raise e

if __name__ == "__main__":
  drive_mount()
""".format(user, user, env['user_home'])

  with open(path_join(env['user_home'], "drive_mount.sh"), "w") as f:
    f.write(mount)

  if debug:
    !stat $user_home
  
  !chown -R  $user $user_home
  !chmod u+x $drive_mount_bin

 # !runuser -l $user -c "mkdir /home/$user/drive" > /dev/null 2>&1
  !runuser -l $user -c "python3 /home/$user/drive_mount.sh"

  !ln -s /content/drive/My\ Drive $user_home/drive > /dev/null 2>&1
  !chown $user:$user $user_home/drive > /dev/null 2>&1

if storage == "Google Drive" or run:
  drive_mount()

### Debug
You won't have fun in here.

In [ ]:
#@title As administrator
#@markdown Check sign-in this cell to sign-in.
mountpoint = "/content/drive" #@param {type:"string"}
signin = False #@param {type:"boolean"}
define = False #@param {type:"boolean"}

if define:
  def drive_mount(mountpoint='/content/drive'):
    from os.path import join
    from google.colab import drive
    drive_root_directory = join(mountpoint, "My Drive")
    try:
      drive.mount(mountpoint)
    except Exception as e:
      raise e
  
if signin:
  drive_mount()

## Persistence

*In here we find ways for this creature to live, prosper and reproduce*.

We have a cron tar every X minutes

In [ ]:
#@title Simple Keep Alive (12 hours) { vertical-output: true }
#@markdown *Press this button from the desktop environment to keep this machine alive for 12 hours.*

#@markdown Basically press `Up` and `Down` every `ping` seconds.
run = False #@param {type:"boolean"}

user = "user" #@param {type:"string"}
# How much time between
ping = 5 #@param {type:"integer"}

busy = True #@param {type:"boolean"}

def simple_keep_alive(user=user, interaction_interval=ping):
  !apt install xdotool xattr > /dev/null 2>&1
  from time import sleep

  from os import environ
  environ['user'] = user

  while True:

    # "Starting Up and down"
    try:
      output = !runuser -l user -c "DISPLAY=:1.0 xdotool key 'Up'"
      assert "Failed creating new xdo instance" in output[0]
      sleep(ping)
      !runuser -l user -c "DISPLAY=:1.0 xdotool key 'Down'"
      sleep(ping)
    except AssertionError as no_session:
      print("You need to run this cell from inside the VNC Session.")
      break

if run:
  simple_keep_alive()

In [ ]:
#@title Restore
#@markdown *Restore `/etc` and `user`'s  home from files.*
run = True #@param {type:"boolean"}

user = "user" #@param {type:"string"}
method = "Google Drive" #@param ["Google Drive", ""] {allow-input: true}

def restore(user=user, method=method):
  from os import environ
  from os import listdir as ls
  from os.path import exists
  environ['user'] = user
  environ['drive_path'] = "/home/{}/drive/home.tar.gz".format(user)

  if method == "Google Drive":

    try:
      exist = !runuser -l $user -c "ls $drive_path"
      assert not "No such file or directory" in exist[0] 

    except AssertionError as no_backup:
      print("First boot, no restore")
      return

    !runuser -l $user -c "cp -r /home/$user/drive/home.tar.gz /home/$user/home.tar.gz" 
    !tar -xpzf /home/$user/home.tar.gz -C / --numeric-owner

    !runuser -l $user -c "cp -r /home/$user/drive/etc.tar.gz /home/$user/etc.tar.gz" 
    !tar -xpzf /home/$user/etc.tar.gz -C / --numeric-owner
  
    !rm /home/$user/etc.tar.gz /home/$user/home.tar.gz

  !service cron restart > /dev/null 2>&1
  !service rsyslog restart > /dev/null 2>&1
  !service ssh restart > /dev/null 2>&1
  !service tor restart > /dev/null 2>&1

if run or storage == "Google Drive":
  restore()


In [ ]:
#@title Backup home, etc to files (`/home/user/drive/`) { vertical-output: true }

#@markdown *In here we backup `user`'s home and other funny things.*

run  = True #@param {type:"boolean"}

#@markdown **Manual intervention required**: timer keeps the  cell running.

busy  = False


user = "user" #@param {type:"string"}

#@markdown #### Storage
method = "Google Drive" #@param ["Google Drive", ""] {allow-input: true}

#@markdown Every:
minutes = 20 #@param {type:"integer"}
#@markdown of every
hours = 1 #@param {type: "integer"}

def keep_alive(user=user, hours=hours, minutes=minutes, method=method, start=True):

  # Env
  from os import environ
  environ['user'] = user
  from time import sleep

  # Time check
  if hours == 0 or minutes == 0:
    print("values have to be positive")
    return

  # Blacklist
  excluded_paths = ["/home/{}/drive".format(user),
                    "/home.tar.gz",
                    "/etc.tar.gz",
                    "/home/{}/home.tar.gz".format(user),
                    "/home/{}/etc.tar.gz".format(user),
                    "/home/{}/.config/Google".format(user),
                    "/home/{}/.local/share/Steam".format(user),
                    ]


  # Exclude section of the command
  add_exclude = lambda path: "--exclude={}".format(path)
  exclude_command = " ".join([add_exclude(path) for path in excluded_paths])

  # Permissions section
  chown = lambda user, path: "chown {}:{} {}".format(user, user, path)
  chown_home = chown(user, "/home.tar.gz")
  chown_etc = chown(user, "/etc.tar.gz")

  # Move Home section
  if method == "Google Drive":
    backup_path = "/home/{}/drive/home.tar.gz".format(user)
  copy_home_backup = lambda user: "runuser -l {} -c 'mv /home.tar.gz {}'".format(user, backup_path)
  copy_home_user = copy_home_backup(user)

  # Move Etc section
  if method == "Google Drive":
    backup_path = "/home/{}/drive/etc.tar.gz".format(user)
  copy_etc_backup = lambda user: "runuser -l {} -c 'mv /etc.tar.gz {}".format(user, backup_path)
  copy_etc_user = copy_etc_backup(user)

  # Complete commands
  backup_home = ("cd / && tar -cpzf home.tar.gz {} --one-file-system /home/{}"
                 " > /dev/null 2>&1 && {} && {}").format(exclude_command, 
                                                         user, 
                                                         chown_home, 
                                                         copy_home_user)

  backup_etc = ("cd / && tar -cpzf etc.tar.gz {} --one-file-system /etc"
          " > /dev/null 2>&1 && {} && {}").format(exclude_command, 
                                                  chown_etc, 
                                                  copy_etc_user)
          
  #def run_now(command):
  #  from os import environ
  #  environ['command'] = command
  #  !$command

  #if start:
  #  run_now()

  # Cron job template
  cron_job = """
PATH=/usr/sbin:/usr/sbin:/usr/bin:/sbin:/bin

# Backup every {} minutes and {}  everyday

# Home
0-59/{} 0-23/{} * * * root {}

# Etc
0-59/{} 0-23/{} * * * root {}

""".format(minutes, hours, 
           minutes, hours, backup_home, 
           minutes, hours, backup_etc)

  # Save cron job
  with open("/etc/cron.d/backup", "w") as f:
    f.write(cron_job)
  !chmod 644 /etc/cron.d/backup


  # Restart affected services
  !service cron restart > /dev/null 2>&1
  !service rsyslog restart > /dev/null 2>&1
  !apt install openssh-server autossh > /dev/null 2>&1
  !service ssh restart  > /dev/null 2>&1

def busy_loop(user=user, minutes=minutes, hours=hours):

  from os import environ
  environ['user'] = user
  from time import sleep

  while True:
   # !runuser -l $user -c "tar -cpzf home.tar.gz --exclude=/home.tar.gz --one-file-system /home/user &"i
   !date
   !echo "backup started"
   !cd / && tar -cpzf home.tar.gz --exclude=/home.tar.gz --exclude=/etc.tar.gz --one-file-system /home/user > /dev/null 2>&1
   !cd / && tar -cpzf etc.tar.gz --exclude=/etc.tar.gz --exclude=/home.tar.gz --one-file-system /etc > /dev/null 2>&1
   !chown $user:$user /home.tar.gz /etc.tar.gz

  
   !runuser -l $user -c "cp /etc.tar.gz /home/$user/drive/etc.tar.gz"
   print("complete, coming back in {} hours and {} minutes.".format(hours, minutes))
   #!runuser -l $user -c "tar -cpzf etc.tar.gz --exclude=/etc.tar.gz --one-file-system /etc"
   sleep(hours + 60*minutes)

if busy and run:
  busy_loop()

if not busy and (run or storage == "Google Drive"):
  keep_alive()

## Connection
*In here we open the machine of this notebook to remote connections.*

**Manual intervention required**: On first run, to configure the tunnel.

Available connection methods are:
- SSH reverse tunnel; 
- Tor.

**Worst case**: 30 seconds

### Routines

In here there are functions needed to set up connection

In [ ]:
#@title Install and configure packages { vertical-output: true }
#@markdown We install `openssh-server`, `autossh`, `tor` and `nmap`
run = True #@param {type:"boolean"}
 
def install_ssh(run=True):
  !killall apt > /dev/null 2>&1
  !rm /var/lib/dpkg/lock-frontend
  !dpkg --configure -a > /dev/null 2>&1
  !apt  install -y openssh-server autossh tor nmap > /dev/null 2>&1

  ssh_config = """
Host *
    ForwardX11 yes
    ForwardX11Trusted yes
    PasswordAuthentication no
    Tunnel yes
    SendEnv LANG LC_*
    HashKnownHosts yes
    GSSAPIAuthentication yes
"""

  sshd_config = """
ChallengeResponseAuthentication no
UsePAM yes
X11Forwarding yes
PrintMotd no
AcceptEnv LANG LC_*
Subsystem	sftp	/usr/lib/openssh/sftp-server
"""
  with open("/etc/ssh/ssh_config", "w") as f:
    f.write(ssh_config)
  with open("/etc/ssh/sshd_config", "w") as f:
    f.write(sshd_config)

  #TODO
  # Create a preconfigured bare torrc file in here

  if run:
    !service ssh start > /dev/null 2>&1
    !service tor start  > /dev/null 2>&1

if run:
  install_ssh()

In [ ]:
#@title Show onion SSH hostname { vertical-output: true }
run = False #@param {type:"boolean"}

def get_tor_ssh_hostname():
  from os.path import exists
  onion_hostname_path = "/var/lib/tor/ssh/hostname"
  if not exists(onion_hostname_path):
    print("Tor still unconfigured")
    return
  with open('/var/lib/tor/ssh/hostname', 'r') as f:
    return f.read()  

if run:
  print(get_tor_ssh_hostname())

In [ ]:
#@title Init SSH directory { vertical-output: true }
#@markdown Create SSH directory for user `user` and `root`
user = "user" #@param {type:"string"}
root = True #@param {type:"boolean"}

def new_ssh_dirs(user=user, root=root):
  from os import environ
  environ['user'] = user
  environ['ssh_dir'] = "/home/{}/.ssh".format(user)
  !mkdir -p /home/$user/.ssh > /dev/null 2>&1
  !chmod 700 /home/$user/.ssh
  !chown user:user /home/$user/.ssh
  
  config = """# Onion support
Host *.onion
      proxyCommand ncat --proxy 127.0.0.1:9050 --proxy-type socks5 %h %p

Host google_shell
      Hostname localhost
      Port 6666
      User user
"""

  with open("/home/{}/.ssh/config".format(user), "w") as f:
    f.write(config)
  !chown -R user:user $ssh_dir

  if root:
    !mkdir -p /root/.ssh > /dev/null 2>&1
    !chmod 700 /root/.ssh

In [ ]:
#@title New SSH key pair { vertical-output: true }

run = False #@param {type:"boolean"}

#@markdown If it's your first run you could need to generate keys
user = "user" #@param ["user"]

def new_pair(user=user):
  from os import environ
  environ['user'] = user
  !runuser -l $user -c 'ssh-keygen'


#@markdown #### **Show fingerprint**
#@markdown RSA public key of user `user`
user = "user" #@param {type:"string"}

def show_fingerprint(user=user):
  from os import environ
  environ['user'] = user
  print("{} ssh public key:".format(user))
  !cat /home/$user/.ssh/id_rsa.pub

  if run:
    new_pair()
    show_fingerprint()

#### Existing configuration
If it's not your first run you want to retrieve your already existing ssh key pairs, to retrieve from a secure location.
We currently have code for:
- Google drive.

You need to have (in the root of your google drive)  
- `/etc/ssh/` properly configured as in a normal ssh server;
- `/etc/tor/` properly containeing a `torrc` with ssh hidden service enabled;
- `/.ssh/` containing a key called `google` and a `config` file contaning an host called `google` to use as a reverse proxy;
- `/var/lib/tor/ssh/`

##### Open from Google **drive**

###### Routines

In [ ]:
#@title Mount
#@markdown Check sign-in this cell to sign-in as `user`.
user = "user" #@param {type:"string"}
run = False #@param {type:"boolean"}
debug = False #@param {type:"boolean"}
mountpoint = "/content/drive"

from os import getuid, getgid, setuid, setgid
from os import environ as env
from os.path import join as path_join
from os import listdir as ls


env['user'] = user
env['user_home'] = "/home/{}".format(user)
env['drive_mount_bin'] = path_join(env['user_home'], 'drive_mount.sh')

def drive_mount():
  !runuser -l $user -c "yes | python3 -m pip install --user google-colab"  > /dev/null 2>&1

  def mount(user=user, debug=debug):
    !cp -r /root/.config/Google /home/$user/.config/Google
    !mkdir -p /home/$user/.config/Google/DriveFS/Logs
    !chown -R $user:$user /content/.config
    !chown -R $user:$user /home/user/.config/Google/DriveFS/Logs

  mount()

  mount = """
#!/bin/sh

from os import environ as env
from os.path import join as path_join
from google.colab import drive

env['CLOUDSDK_CONFIG']  = '/content/.config'

def drive_mount(mountpoint=path_join('/home/{}', 'drive'), user="{}"):
  print(drive._env)
  print(drive._os.environ['HOME'])
  #print(drive._os.environ['CLOUDSDK_CONFIG'])
  #  drive_root_directory = path_join({}, "drive")
  #  home_content = ls(env['user_home'])
  try:
    drive.mount(mountpoint)
  except Exception as e:
    raise e

if __name__ == "__main__":
  drive_mount()
""".format(user, user, env['user_home'])

  with open(path_join(env['user_home'], "drive_mount.sh"), "w") as f:
    f.write(mount)

  if debug:
    !stat $user_home
  
  !chown -R  $user $user_home
  !chmod u+x $drive_mount_bin

  !runuser -l $user -c "mkdir /home/$user/drive" > /dev/null 2>&1
  !runuser -l $user -c "python3 /home/$user/drive_mount.sh"

if run:
  drive_mount()
  !echo $CLOUDSDK_CONFIG

In [ ]:
#@title Copy from or to root
user = "user" #@param {type:"string"}
source = "/home/user/drive/.ssh" #@param {type:"string"}
destination = "/home/user/" #@param {type:"string"}
run = False #@param {type:"boolean"}

from os import environ

!rm -rf /home/user/.ssh

def drive_copy(source=source, dest=destination):
  from os.path import join as path_join
  environ['source'] = source
  environ['dest'] = dest
  environ['user'] = user
  environ['temp'] = path_join("/home/{}".format(user), ".tmp", source)

  #!rm -r /home/$user/.tmp
  #!runuser -l $user -c "mkdir -p $temp"
  #!ls /home/$user/.tmp
 # !rm -r /tmp$dest
  #!runuser -l $user -c "mkdir  -p $tmp$source"
  !runuser -l $user -c "cp -a $source $temp"
  
  !cp -rv $temp $dest

if run:
  drive_copy()
  !ls /home/user/.ssh

In [ ]:
#@title Import
#@markdown We import:


key = "google" #@param ["id_rsa", "google"] {allow-input: true}
root = True #@param {type:"boolean"}
user = "user" #@param {type:"string"}

def drive_download(key=key, user=user, root=root):
  from os import environ
  environ['key'] = key
  environ['user'] = user
  #@markdown - SSH daemon configuration;

  #@markdown - user configuration;
  !runuser -l $user -c "mkdir -p /home/$user/.tmp"
  
  !runuser -l $user -c "cp -r /home/$user/drive/.ssh /home/$user"

  !runuser -l $user -c "cp -r '/home/$user/drive/etc' '/home/$user/.tmp'"
  !cp -r /home/$user/.tmp/etc /

  !mv /home/$user/.ssh/$key /home/$user/.ssh/id_rsa
  !mv /home/$user/.ssh/$key.pub /home/$user/.ssh/id_rsa.pub
  !cp /home/$user/.ssh/id_rsa.pub /home/$user/.ssh/authorized_keys

  #@markdown - root user configuration;
  !cp /home/$user/.ssh/id_rsa.pub /root/.ssh/authorized_keys

  # Set permissions
  !chmod 700 /home/$user/.ssh
  !chmod 600 /home/$user/.ssh/id_rsa /home/$user/.ssh/id_rsa.pub
  !chmod 755 /home/$user/.ssh/authorized_keys

  !chown -R $user:$user /home/$user

  #@markdown - tor configuration

  !runuser -l $user -c "cp -r '/home/$user/drive/var' '/home/$user/.tmp'"
  !cp -r /home/$user/.tmp/var /
  !chown -R debian-tor:debian-tor /var/lib/tor

  #  !!runuser -l $user -c "cp -r '/home/$user/drive/var/' '/home/$user/.tmp/var'"
  #!cp -r /home/$user/.tmp/var /var
  #!cp -r /content/drive/My\ Drive/etc/tor /etc/tor
  #!cp -r /content/drive/My\ Drive/var/lib/tor/ssh /var/lib/tor/

  # Restart services
  !service ssh restart > /dev/null 2>&1
  !service tor restart > /dev/null 2>&1

In [ ]:
#@title Export
#@markdown - SSH daemon configuration;
#@markdown - Tor configuration and onion address private key;
#@markdown - user configuration.
root = True #@param {type:"boolean"}
user = "user" #@param {type:"string"}

def drive_upload(user=user, root=root):
  from os import environ
  environ['user'] = user
  # System-wide SSH config files
  !mkdir -p /content/drive/My\ Drive/etc/ssh
  !cp /etc/ssh/sshd_config /content/drive/My\ Drive/etc/ssh/sshd_config
  !cp /etc/ssh/ssh_config /content/drive/My\ Drive/etc/ssh/ssh_config

  # SSH user directory
  !cp -r /home/$user/.ssh "/content/drive/My Drive"

  # Tor configuration and private keys
  !mkdir -p /content/drive/My\ Drive/var/lib/tor
  !mkdir -p /content/drive/My\ Drive/etc/tor
  !cp -r /var/lib/tor/ssh /content/drive/My\ Drive/var/lib/tor/ssh
  !cp -r /etc/tor/torrc /content/drive/My\ Drive/etc/tor

##### Run

In [ ]:
#@title ### **Migration tool**
#@markdown  
mountpoint = "/content/drive" #@param {type:"string"}
action = "Import" #@param ["Import", "Export", "Just mount"]
run = False #@param {type:"boolean"}


def migration_tool(mountpoint=mountpoint, action=action, debug=run):
  from os.path import join as path_join

#  def drive_mount(mountpoint='/content/drive'):
#    from google.colab import drive
#    drive_root_directory = path_join(mountpoint, "My Drive")
#    try:
#      drive.mount(mountpoint)
#    except Exception as e:
#      raise e
    
  #drive_mount()

  if action == "Import":
    drive_download()
  if action == "Export":
    drive_upload()

  if debug:
    show_fingerprint()
    print("ssh onion address:")
    !cat /var/lib/tor/ssh/hostname

  with open('/var/lib/tor/ssh/hostname', 'r') as f:
    return f.read()

if run:
  migration_tool()

##### Debug

In [ ]:
#@title Mount (as administrator)
#@markdown Check sign-in this cell to sign-in.
mountpoint = "/content/drive" #@param {type:"string"}
run = False #@param {type:"boolean"}

def drive_mount(mountpoint='/content/drive'):
  from os.path import join
  from google.colab import drive
  drive_root_directory = join(mountpoint, "My Drive")
  try:
    drive.mount(mountpoint)
  except Exception as e:
    raise e
  
if run:
  drive_mount()

### Run

*In here the connection takes place (and reverse connection parameters are set).*

In [ ]:
#@title Set reverse proxy { vertical-output: true }
#@markdown *In here we configure a reverse proxy*

run = False #@param {type:"boolean"}

#@markdown #### **Local configuration** (*the machine on this notebook*)
#@markdown The user which will open the connection:
user = "user" #@param {type:"string"}
#@markdown #### **Remote configuration** (*your computer or a proxy*)
remote_user = "google" #@param {type:"string"}
domain = "extranet.arcipelago.ml" #@param {type:"string"}
port = 22 #@param {type:"integer"}

def create_ssh_config(user=user, remote_user=remote_user, domain=domain, port=port):
  config = """# Onion support
Host *.onion
      proxyCommand ncat --proxy 127.0.0.1:9050 --proxy-type socks5 %h %p

Host google
      HostName {}
      Port {}
      User {}

Host google_shell
      Hostname localhost
      Port 6666
      User user
""".format(domain, port, remote_user)
  with open("/home/{}/.ssh/config".format(user), "w") as f:
    f.write(config)

if run:
  create_ssh_config()


In [ ]:
#@title Connect { vertical-output: true }
#@markdown If this is your first setup, select *new*:
keys = "existing" #@param ["new", "existing"]
#@markdown Select this option if you want to enable *root* access.
root = True #@param {type:"boolean"}
#@markdown Specify the user do you want to access to.
user = "user" #@param {type:"string"}
#@markdown Enable if you wish to assign an onion address to this machine.
tor = True #@param {type:"boolean"}
#@markdown Enable if you wish to connect to the machine through a reverse proxy (see above).
reverse = True #@param {type:"boolean"}
local = 22 #@param {type:"integer"}
remote = 6666 #@param {type:"integer"}

from os import environ
environ['user'] = user
environ['local_port'] = str(local)
environ['remote_port'] = str(remote)

install_ssh()
new_ssh_dirs()
if keys == "new":
  new_pair(user)
  if root:
    new_pair("root")
  migration_tool(mountpoint=mountpoint, action="Export")
if keys == "existing":
  migration_tool(mountpoint=mountpoint, action='Import')

  print("""you can connect to
   {}@{}""".format(environ['user'], get_tor_ssh_hostname()))

if reverse:
  !ssh -tt -o StrictHostKeyChecking=no -F /home/user/.ssh/config -i /home/user/.ssh/id_rsa -R 6666:localhost:22 google 'exit' >/dev/null 2>&1
  !autossh -N -M 10984 -o 'PubkeyAuthentication=yes' -o 'PasswordAuthentication=no' -o StrictHostKeyChecking=no -F /home/$user/.ssh/config -i /home/$user/.ssh/id_rsa -f -Y -R $remote_port:localhost:$local_port google
  print("""if you connected with a reverse proxy, you can to
     {}@localhost, port {}""".format(user, remote))

### Debug

*In here there are functions to discover what's going on while trying to setup availability for remote connections.*

In [ ]:
#@title Test SSH connection
#@markdown It connects to hostname `google`. 
start = False #@param {type:"boolean"}

if start:
  !ssh -o StrictHostKeyChecking=no -F /home/user/.ssh/config -i /home/user/.ssh/id_rsa google
  # Verboso
  # !ssh -i /home/user/.ssh/id_rsa -o StrictHostKeyChecking=no -p 58372 google@extranet.arcipelago.ml

In [ ]:
#@title Test SSH reverse tunnel
#@markdown it opens a reverse tunnel to hostname `google`.
run = False #@param {type:"boolean"}


if run:
  !ssh -o StrictHostKeyChecking=no -F /home/user/.ssh/config -i /home/user/.ssh/id_rsa -R 6666:localhost:22 google

In [ ]:
#@title Show ssh config
#@markdown it shows the ssh configuration file for an user
user = "user" #@param {type:"string"}
#@markdown (activate the switch before running it)
start = 0 #@param {type:"slider", min:0, max:1, step:1}

from os import environ

environ['user'] = user

if start:
  if user != "root":
    !cat /home/$user/.ssh/config
  if user == "root":
    !cat /root/.ssh/config

In [ ]:
#@title Show SSH directory
#@markdown show the content of the `.ssh` directory of an user.

user = "user" #@param {type:"string"}
print_tty = False #@param {type:"boolean"}
start = False #@param {type:"boolean"}
variable_name = False
variable_name = ""

if start:
  if user != "root":
    !ls -lsh /home/user/.ssh
    !du /home/user/.ssh
    !stat /home/user/.ssh
  if user == "root":
    !ls -lsh /root/.ssh
    !du /root/.ssh
    !stat /root/.ssh

In [ ]:
#@title Show `user` known_hosts
start = False #@param {type:"boolean"}
if start:
  !cat /home/user/.ssh/known_hosts

In [ ]:
#@title Copy `user` SSH keys on proxy
start = False #@param {type:"boolean"}

if start:
  !scp -i /home/user/.ssh/id_rsa -o StrictHostKeyChecking=no -F /home/user/.ssh/config /home/user/.ssh/id_rsa* google:~/.ssh/

In [ ]:
#@title Check if /dev/tty exists
run = False #@param {type:"boolean"}

if run:
  !ls -la /dev/tty

In [ ]:
#@title Show private key of user `user`
run = False #@param {type:"boolean"}

if run:
  !cat /home/user/.ssh/id_rsa

In [ ]:
#@title Setup **tunnel device** (user level)
#@markdown Create a tunnel device for user `user`
user = "user" #@param {type:"string"}
run = False #@param {type:"boolean"}

if run:
  from os import environ

  environ['user'] = user
  !ip tuntap add name tun0 mode tun user $user
  !ip address add 192.0.2.10/24 dev tun0
  !ip link set dev tun0 up

## Graphical environment

*In here we install an user-friendly graphical environment to easy advanced tasks and setup a screen sharing program.*

**Manual intervention required**: at first boot, to set VNC password at runtime.

If you enabled Google Drive integration you will find a link to your drive in your home directory.

**XFCE (default)**:
- **Best**: 9 mins
- **Worst**: 12 mins

**MATE**:
- **Best**:
- **Worst**:  4 mins

### Routines

*In here we provide functions to install and configure TigerVNC and additional tools useful in a desktop.*

In [ ]:
#@title Install and configure TigerVNC { vertical-output: true }
#@markdown I chose TigerVNC because it was the easiest to configure.
from os import environ
run = True #@param {type:"boolean"}


def setup_vnc():
  #@markdown - We install the following packages:
  #@markdown `dbus-x11 tigervnc-server-standalone tigervnc-xorg-extension xinit xserver-xorg-video-dummy` `x11-xserver-utils` `xauth`;
  !apt --quiet update > /dev/null 2>&1
  !apt --fix-broken install > /dev/null 2>&1
  !rm /var/lib/dpkg/statoverride
  !apt --quiet install dbus-x11 tigervnc-standalone-server tigervnc-xorg-extension xinit xserver-xorg-video-dummy x11-xserver-utils xauth  > /dev/null 2>&1

  #@markdown 1. Setup a simple xorg  video dummy configuration.

  #@markdown 2. Create an Xauthority file

  #@markdown 3. Create an Xresources file

  #@markdown 4. Allow any user to start X

  #@markdown 5. Set VNC credentials:
  user = "user" #@param {type:"string"}
  password = "testone" #@param {type:"string"}


  # 1
  !wget http://xpra.org/xorg.conf > /dev/null 2>&1
  !cp xorg.conf /etc/X11

  # 2
  !runuser -l $user -c 'touch /home/$user/.Xauthority'

  # 3
  !runuser -l $user -c "touch /home/$user/.Xresources"

  # 4
  xwrapper = "allowed_users=anybody"
  with open("/etc/X11/Xwrapper.config", 'w') as f:
    f.write(xwrapper)

  # Set vnc for user
  !runuser -l $user -c "mkdir -p /home/$user/.vnc"
  !runuser -l $user -c "mkdir -p /home/$user/drive/.vnc"

  # 5
  environ['vnc_password'] = password
  environ['vnc_password_path'] = "/home/{}/.vnc/passwd".format(user)

  # Create password
  !runuser -l $user -c "echo $vnc_password | vncpasswd -f > $vnc_password_path"
  # Set config
  environ['vnc_config_path'] =  "/home/{}/.vnc/config".format(user)

  config = """session=xfce
geometry=1920x1080
localhost
alwaysshared"""

  with open(environ['vnc_config_path'], "w") as f:
    f.write(config)

  # Permission check
  !chown -R $user:$user /home/$user

    #!runuser -l $user -c "echo $vnc_config > $vnc_config_path" 
    #!vim $vnc_password_path

if run:
  setup_vnc()

In [ ]:
#@title Set VNC Startup file
#@markdown *Here we set the VNC startup file*
run = False #@param {type:"boolean"}

#@markdown What session do you want?
session = "mate-session" #@param {type:"string"}

def set_vnc_startup_file(user=user, session=session):
  from os import environ
  environ['user'] = user
 
  # Set VNC startup file
  !mkdir -p /home/$user/.vnc
  !chown $user:$user /home/$user/.vnc

  xstartup = """#!/bin/sh
unset SESSION_MANAGER
unset DBUS_SESSION_BUS_ADDRESS
[ -x /etc/vnc/xstartup ] && exec /etc/vnc/xstartup
[ -r $HOME/.Xresources ] && xrdb $HOME/.Xresources
vncconfig -iconic &
/usr/bin/{}
""".format(session)

  with open("/home/{}/.vnc/xstartup.new".format(user), "w") as f:
    f.write(xstartup)

  !chown $user:$user /home/$user/.vnc/xstartup.new
  !chmod 755 /home/$user/.vnc/xstartup
  !runuser -l $user -c "mv /home/$user/.vnc/xstartup.new /home/$user/.vnc/xstartup"
  !chmod 644 /home/$user/.vnc/xstartup


if run:
  set_startup_file()

In [ ]:
#@title Install Desktop Environment { vertical-output: true }
#@markdown *This lets you install a DE*

run = False #@param {type:"boolean"}

#@markdown What you can choose:
DE = "MATE" #@param ["MATE", "XFCE"]
install_type = "full" #@param ["minimal",  "full"]

def record(variable, env_name): 
  if variable: 
    environ[env_name] = variable
    return True

def de_install(DE=DE, install_type=install_type):
  if DE == "MATE":
    if install_type == "minimal":
      packages = ("mate-panel marco mate-session-manager"
      "mate-control-center" "mate-applets" "fonts-dejavu")
      record(packages, "packages")
      !apt install $packages
    if install_type == "full":
      !apt --quiet install -y mate > /dev/null 2>&1
    session = "mate-session"

  if DE == "XFCE":
    !apt --quiet install -y xubuntu-desktop > /dev/null 2>&1
    session = "xfce4-session"

  return session

if run:
  session = de_install()

In [ ]:
#@title Application preferences { vertical-output: true }
run = True #@param {type:"boolean"}

#@markdown What's a VNC server without a VNC client?
vnc_viewer = "remmina" #@param ["remmina", ""] {allow-input: true}

#@markdown Your web browser of choice (chromium is pre-installed).
web_browser = "qutebrowser" #@param ["firefox", "epiphany-browser", "qutebrowser", "qutebrowser_pip", "epiphany-browser", "chromium-browser"] {allow-input: true}

#@markdown Install an email client
email = "evolution" #@param ["evolution", "thunderbird", ""]

#@markdown Do you want to chat?
chat = True #@param {type:"boolean"}

#@markdown Do you want to play games?
games = "steam" #@param ["steam", "gnome-games", "epsxe", ""]

#@markdown Your favorite media player.
media_player = "mpv" #@param ["mpv", "vlc", "totem"] {allow-input: true}

#@markdown A WSIWYG text editor
text_editor = "gedit" #@param ["thunar", "caja", ""] {allow-input: true}

#@markdown Do you want to edit images?
image_editing = True #@param {type:"boolean"}

#@markdown A file manager
file_manager = "nautilus" #@param ["gnome-builder", "kate", ""] {allow-input: true}

#@markdown Sticky notes
sticky_notes = "tomboy" #@param ["tilix", ""] {allow-input: true}

#@markdown System monitoring (*and Usage and Baobab*)
monitoring = "gnome-system-monitor" #@param [""] {allow-input: true}

#@markdown A terminal emulator
terminal = "tilix" #@param ["tilix", ""] {allow-input: true}

#@markdown A visual package manager
package_manager = "synaptic" #@param ["synaptic", "gnome-packagekit", "gnome-software"] {allow-input: true}

#@markdown A users utility
users = "gnome-system-tools" #@param ["gnome-system-tools", ""]

#@markdown Visual style
style = "gnome" #@param ["gnome", ""] {allow-input: true}

#@markdown Install tweaks
tweaks = True #@param {type:"boolean"}

#@markdown Install dock
dock = False #@param {type: "boolean"}

def record(variable, env_name): 
  if variable: 
    environ[env_name] = variable
    return True

def install_steam():
  # Enable multiverse
  !add-apt-repository multiverse > /dev/null 2>&1
  # Enable i386 arch
  !dpkg --add-architecture i386
  !apt update > /dev/null 2>&1
  !apt --fix-broken install > /dev/null 2>&1 #install(package=package, system=system):

  # Install zenity
  !apt install zenity > /dev/null 2>&1
  # LibGL
  !apt install libgl1:i386 > /dev/null 2>&1
  # Nvidia thing
  output = !apt install libgl1-nvidia-glvnd-glx:i386
  print(output)

 # Get and install steam
  !cd /tmp && wget https://cdn.cloudflare.steamstatic.com/client/installer/steam.deb > /dev/null 2>&1
  !dpkg -i /tmp/steam.deb > /dev/null 2>&1

!dpkg --configure -a > /dev/null 2>&1

if record(vnc_viewer, "vnc_viewer"):
  !apt install $vnc_viewer > /dev/null 2>&1

if record(web_browser, "web_browser"):
  if web_browser == "qutebrowser_pip":
    !runuser -l $user -c "python3 -m pip install --user qutebrowser"
    !runuser -l $user -c "export PATH=/home/$user/.local/bin:$PATH"
  else:
    !apt install $web_browser > /dev/null 2>&1
  !apt install chromium-browser > /dev/null 2>&1

if record(email, "email"):
  !apt install $email > /dev/null 2>&1

if chat:
  !apt install gajim telegram-desktop xchat > /dev/null 2>&1
  # Add upstream and fractal

if record(games, "games"):
  if games == "steam":
    install_steam()
  else:
    !apt install $games

if record(media_player, "media_player"):
  !apt install $media_player > /dev/null 2>&1
  !apt install pavucontrol > /dev/null 2>&1

if record(file_manager, "file_manager"):
  !apt install $file_manager > /dev/null 2>&1

if record(text_editor, "text_editor"):
  !apt install $text_editor > /dev/null 2>&1

if record(sticky_notes, "sticky_notes"):
  !apt install $sticky_notes > /dev/null 2>&1

if record(monitoring, "system_monitor"):
  !apt install $system_monitor gnome-usage baobab file-roller  > /dev/null 2>&1

if record(terminal, "terminal"):
  !apt install $terminal > /dev/null 2>&1

if record(package_manager, "package_manager"):
  package_manager = "{} gdebi".format(package_manager)
  record(package_manager, "package_manager")
  !apt install $package_manager > /dev/null 2>&1

if record(users, "users"):
  !apt install intltool > /dev/null 2>&1
  !apt install $users > /dev/null 2>&1

if record(style, "style"):
  if style == "gnome":
    style = ("gnome-icon-theme adwaita-icon-theme"
    "adwaita-icon-theme-full" 
    "gnome-themes-standard fonts-cantarell")
    record(style, "style")
  !apt install $style > /dev/null 2>&1

if dock:
  !git clone https://github.com/ubuntu-mate/mate-dock-applet > /dev/null 2>&1
  !apt install build-essentials > /dev/null 2>&1
  !cd mate-dock-applet && aclocal > /dev/null 2>&1
  !cd mate-dock-applet && automake --add-missing > /dev/null 2>&1
  !cd mate-dock-applet && autoreconf > /dev/null 2>&1
  !cd mate-dock-applet && ./configure --prefix=/usr --with-gtk3 > /dev/null 2>&1
  !cd mate-dock-applet && make > /dev/null 2>&1 
  !cd mate-dock-applet && make install > /dev/null 2>&1

if tweaks:
  trigger = "apt --fix-broken install"
  tweak_packages = ("gnome-tweak-tool mate-tweak" 
    "mate-dock-applet libnotify-bin dconf-editor"
    "xdg-user-dirs-gtk")
  record(tweak_packages, "tweak_packages")
  output = !apt install $tweak_packages > /dev/null 2>&1
  for line in output:
    if trigger in line:
      !apt --fix-broken install

In [ ]:
#@title Colab setup{ vertical-output: true }
#@markdown *Get the link for a colab file on google drive and eventually open it on startup.*

run = True #@param {type: "boolean"}

user = "user" #@param {type:"string"}
path = "/home/user/drive/colab_desktop/gnucolab.ipynb" #@param {type:"string"}

#@markdown Enable the following checkbox if you want to open the notebook on session start:

autostart = True #@param {type: "boolean"}

def get_link(user=user, path=path, autostart=autostart):
  from os import environ
  environ['user'] = user
  environ['drive_file_path'] = path

  fid = !runuser -l $user -c "xattr -p 'user.drive.id' $drive_file_path"

  return "https://colab.research.google.com/drive/{}".format(fid[0])

def autostart_notebook(user=user, link=get_link()):
  from os import environ
  environ['user'] = user

  colab_autostart = """[Desktop Entry]
Type=Application
Name=Colab
Exec=sh -c "sensible-browser {}"
Icon=
Comment=Open a predefined notebook at session signin.
X-GNOME-Autostart-enabled=true""".format(link)

  !mkdir -p /home/$user/.config/autostart

  with open("/home/{}/.config/autostart/colab.desktop".format(user), "w") as f:
    f.write(colab_autostart)

  !chmod +x /home/$user/.config/autostart/colab.desktop
  !chown $user:$user /home/$user/.config
    
if run:
  link = get_link()
  if autostart:
    autostart_notebook()


### Run

*In here we will install DE and run the VNC server.*

**Manual intervention required**: on first run you have to input the VNC password.

### Connection

- VNC;
- Anydesk (see inside).

VNC connection happens through ssh tunnel, so you have to open one on your machine

```console
ssh google_shell -L 9901:localhost:5901
```

and connect with your VNC viewer of choice to your local `9901` port:

```console
 vncviewer localhost:9901
 ```
**Warning:** Increase the port to 5902, 5903, etc in case the VNC server doesn't starts on default display. 
iIt can happen when you run this notebook with a GPU.


In [ ]:
#@title Start VNC server { vertical-output: true }
#@markdown *In here we run tigerVNC server*.
user = "user" #@param {type:"string"}
password = "testone" #@param ["\u003Cauth key>", "testone"] {allow-input: true}
DE = "MATE" #@param ["MATE", "XFCE"]
size = "800x600" #@param ["800x600", "1280x720", "1920x1080"] {allow-input: true}

from os import environ
from os import listdir as ls
environ['user'] = user
environ['size'] = size
environ['display'] = ":1"
environ['vnc_drive_path'] = "/home/{}/drive/.vnc".format(user)
environ['vncserver_options'] = "-geometry {} -alwaysshared".format(size)
environ['password_path'] = "/home/{}/.vnc/passwd".format(user)
environ['xstartup_path'] = "/home/{}/.vnc/xstartup".format(user)
environ['vncserver'] = "vncserver"
environ['vncpasswd'] = password

# DE Installation
session = de_install()
!runuser -l $user -c "xdg-user-dirs-update"
!runuser -l $user -c "xdg-user-dirs-gtk-update"
!apt clean

# Set VNC startup file
set_vnc_startup_file(user, session)

# Set VNC password
!runuser -l $user -c "cp -r $password_path '$vnc_drive_path/passwd'" > /dev/null 2>&1

# Restart procedure
restart = True #@param {type:"boolean"}

if restart:
  restart = !killall vncserver
  restart2 = !killall Xtigervnc

# Not used
# is_vnc_like_process = lambda line: line.startswith('user') and '/usr/bin/vncserver' 

# def get_bunch_of_processes(): 
#   lines = !ps aux | grep vncserver
#   return lines

# vnc_like_processes = lambda: [line for line in get_bunch_of_processes() if is_vnc_like_process(line)]

# if vnc_like_processes():
#   print(len(vnc_like_processes()))

# Start

def run_vnc(user=user, password=password):

  # With password file (not working)
  if password == "\u003Cauth key>":
    !runuser -l $user -c "vncserver -geometry $size -PasswordFile $password_path -alwaysshared  -dpi 96 -localhost :1 > /dev/null 2>&1 &"


  # With text password prompt
  else:
    # Try to open tunnel for vNC on reverse proxy machine
    !runuser -l $user -c "ssh google 'autossh -N -M 10984 -o 'PubkeyAuthentication=yes' -o 'PasswordAuthentication=no' -o StrictHostKeyChecking=no -f -Y -R 5901:localhost:9901 google_shell'"
 
    # Create password if not existing
    if not "passwd" in ls("/home/{}/.vnc".format(user)):
      !runuser -l $user -c "vncpasswd"

    # Run
    !runuser -l $user -c "vncserver -geometry $size -alwaysshared  -dpi 96 -localhost :1 > /home/$user/vnc.log &"

    # Trying to automatize password insertion:
    # !echo -e "$vncpasswd\n$vncpasswd" | vncpasswd -F"

    # Debug (print log)
    debug = True #@param {type:"boolean"}
  
    if debug:
      from time import sleep
      sleep(1)
      !runuser -l $user -c "cat /home/$user/vnc.log"

  #def get_pid(line): return line.split(" ")[:16][-1]

  #from pprint import pprint
  #easy_get_column = lambda line, len: pprint([(i, c) for i, c in enumerate(line.split(" ")[:len])])

  #for line in vnc_like_processes():
  #  easy_get_column(line, 20)
  #print(easy_get_column(line, 20))
  #for line in vnc_like_processes():
  #  print(line)
    #for i, content in easy_get_column(line, 20):
      #print(i, content)
  #  pid = get_pid(line)
  #  print(get_pid(line))
  #  environ['pid'] = pid
  #  print(pid)
  #  !echo $pid
  #  !kill -9 $pid

   #if vnc_like_processes():
    #  print("more than one instance open")
    #  print(vnc_like_processes())
    #for line in vnc_like_processes():
    #  from os import environ
    #  environ['pid'] = get_pid(line)
    #  !kill $pid
  
if run:
  run_vnc()
  

In [ ]:
#@title Simple Keep Alive (12 hours) { vertical-output: true }
#@markdown *Press this button from the desktop environment to keep this machine alive for 12 hours.*

#@markdown Basically press `Up` and `Down` every `ping` seconds.
run = False #@param {type:"boolean"}

user = "user" #@param {type:"string"}
# How much time between
ping = 5 #@param {type:"integer"}

busy = True #@param {type:"boolean"}

def simple_keep_alive(user=user, interaction_interval=ping):
  !apt install xdotool xattr > /dev/null 2>&1
  from time import sleep

  from os import environ
  environ['user'] = user

  while True:

    # "Starting Up and down"
    try:
      output = !runuser -l user -c "DISPLAY=:1.0 xdotool key 'Up'"
      assert "Failed creating new xdo instance" in output[0]
      sleep(ping)
      !runuser -l user -c "DISPLAY=:1.0 xdotool key 'Down'"
      sleep(ping)
    except AssertionError as no_session:
      print("You need to run this cell from inside the VNC Session.")
      break

if run:
  simple_keep_alive()

### Debug

*In here we try other DEs, VNC servers or other connection methods.*

In [ ]:
#@title Install anydesk
#@markdown **Beware**: this program is closed source and *should not* be installed on a production server.
run = False #@param {type:"boolean"}

#@markdown To actually run the program you need to get the key from the UI and then enable the program at the startup.
#@markdown.So you need to connect with VNC first and then enable it.

if run:
  !wget -qO - https://keys.anydesk.com/repos/DEB-GPG-KEY | apt-key add -
  !echo "deb http://deb.anydesk.com/ all main" > /etc/apt/sources.list.d/anydesk-stable.list
  !apt update  > /dev/null 2>&1
  !apt install anydesk > /dev/null 2>&1


In [ ]:
#@title Install (whole) GNOME { vertical-output: true }
#@markdown *You tried it, kid.*
from os import environ
run = False #@param {type:"boolean"}

# package = "gnome" #@param {type:"string"}
type = "minimal" #@param ["minimal", "full"] {allow-input: true}
environ['package'] = package

if type == "minimal":
  environ['package'] = "gnome-shell gnome-session"

if type == "full":
  environ['package'] = "gnome"

if run:
  !apt --quiet update > /dev/null 2>&1
  !apt --quiet install $package # > /dev/null 2>&1

In [ ]:
#@title Install and configure Vino (incomplete)
run = False #@param {type:"boolean"}
#@markdown *Why not?*

if run:
  from os import environ

  #@markdown * We install `dbus-x11 vino xinit xserver-xorg-video-dummy` `x11-xserver-utils` `xauth`;
  !apt --quiet update > /dev/null 2>&1
  !apt --quiet install dbus-x11 vino xinit xserver-xorg-video-dummy x11-xserver-utils xauth > /dev/null 2>&1

  #@markdown - We set an autostart desktop file for vino (`/home/user/.config/autostart`).
  !runuser -l user -c "mkdir -p ~/.config/autostart"

  vino_autostart = """[Desktop Entry]
  Type=Application
  Name=Vino VNC server
  Exec=/usr/lib/vino/vino-server
  NoDisplay=true"""

  with open("/home/user/.config/autostart/vino-server.desktop", 'w') as f:
    f.write(vino_autostart)

  #@markdown - We disable Vino auth prompt
  !runuser -l user -c "dbus-launch gsettings set org.gnome.Vino prompt-enabled false"

  #@markdown We set a VNC password:
  password = "test" #@param {type:"string"}

  environ['password'] = password

  !runuser -l user -c "dbus-launch gsettings set org.gnome.Vino vnc-password $(echo -n $password|base64)"


## Build an Archlinux `chroot`

Basically we have to install `pacman` and `pacstrap` (WIP).

### Routines

In [ ]:
#@title Install `zst`
run = False #@param {type:"boolean"}
version = "latest" #@param ["distro", "latest"]


def install_zst(error=None, version=version):

  !zstd --version

  !apt-get remove zstd > /dev/null 2>&1

  print("upgrading to zst {}".format(version))

  if version == "latest":
    !rm -rf ~/zstd
    !cd ~ && git clone https://github.com/facebook/zstd > /dev/null 2>&1
    !cd ~/zstd && PREFIX=/usr make > /dev/null 2>&1
    !cd ~/zstd && PREFIX=/usr make install  > /dev/null 2>&1
  else:
    !apt install libzstd-dev zstd > /dev/null 2>&1

  !zstd --version

if run:
  install_zst()

In [ ]:
#@title Install `libarchive` { vertical-output: true }
run = False #@param {type:"boolean"}
version = "3.4.3" #@param {type:"string"}

def install_libarchive(error=None, version=version):

  installed = !apt show bsdtar | grep Version

  install_zst()

  print("installing libarchive {}".format(version))

  !rm -rf libarchive
  record(version, "_ver")

  # Download and extract release
  !rm -rf ~/libarchive-$_ver
  !cd ~ && wget https://github.com/libarchive/libarchive/releases/download/v$_ver/libarchive-$_ver.tar.gz > /dev/null 2>&1
  !cd ~ && tar -xzf libarchive-$_ver.tar.gz

  # Build from master (can't set prefix)
  # !cd libarchive-$_ver && PREFIX=/usr cmake . > /dev/null 2>&1
  
  !cd ~/libarchive-$_ver && ./configure --prefix=/usr | grep zst #> /dev/null 2>&1
  !cd ~/libarchive-$_ver && make #> /dev/null 2>&1
  !cd ~/libarchive-$_ver && make install #> /dev/null 2>&1
  !stat /usr/lib/libarchive.so.17
  !ln -s /usr/lib/libarchive.so /usr/lib/libarchive.so.17

  !bsdtar --version
  !tar --version


if run:
  !tar --version 
  install_libarchive()
  !bsdtar --version
 

In [ ]:
#@title Build `arch-install-scripts`
run = False #@param {type:"boolean"}

def build_arch_install_scripts(error=None):
  package = "arch-install-scripts"
  record(package, "pkg")
  !rm -rf ~/$pkg
  !git clone https://git.archlinux.org/$pkg.git ~/$pkg > /dev/null 2>&1
  !cd ~/$pkg && make > /dev/null 2>&1
  !chmod +x ~/$pkg/pacstrap

if run:
  build_arch_install_scripts()
  !~/$pkg/pacstrap --help

In [ ]:
#@title Install pacman { vertical-output: true }
run = False #@param {type:"boolean"}
build_type = "static + dynamic" #@param ["static + dynamic", "dynamic"] {allow-input: true}
version = "5.2.2" #@param {type:"string"}

def install_pacman(error=None, build_type=build_type, version=version):
 
  if build_type == "static + dynamic":
    print("installing pacman")
    record(version, "ver")
    !rm -rf ~/pacman-$ver

    !apt install libarchive-dev > /dev/null 2>&1

    !cd ~ && wget https://sources.archlinux.org/other/pacman/pacman-$ver.tar.gz > /dev/null 2>&1
    !cd ~ && tar -zxf pacman-$ver.tar.gz > /dev/null 2>&1
    configure_options = ("--prefix=/usr --sysconfdir=/etc --localstatedir=/var "
                         "--enable-doc --with-scriptlet-shell=/usr/bin/bash "
                         "--with-ldconfig=/usr/bin/ldconfig")
    record(configure_options, "opts")
    !cd ~/pacman-$ver && ./configure $opts > /dev/null 2>&1
    !cd ~/pacman-$ver && make V=1 > /dev/null 2>&1
    !cd ~/pacman-$ver && make install > /dev/null 2>&1

    !cd ~ && wget https://pkgbuild.com/~eschwartz/repo/x86_64-extracted/pacman-static > /dev/null 2>&1
    !cd ~ && chmod +x pacman-static 
    !cd ~ && cp pacman-static /usr/bin/pacman 
  !pacman --version

if run:
  install_pacman()

In [ ]:
#@title Get `pacman` mirrors
run = False #@param {type:"boolean"}

def pacman_get_mirrors(error=None):
  !mkdir /etc/pacman.d
  !wget https://www.archlinux.org/mirrorlist/all/ -O /etc/pacman.d/mirrorlist > /dev/null 2>&1
  
if run:
  pacman_get_mirrors()

In [ ]:
#@title Enable `pacman` mirrors
run = False #@param {type:"boolean"}

def pacman_enable_mirrors(error=None):
  replace_in_file("/etc/pacman.d/mirrorlist", 
                  "#Server = http://", 
                  "Server = http://")
  #replace_in_file("/etc/pacman.d/mirrorlist",
  #                "#Server = http://mirror.rackspace.com/archlinux/$repo/os/$arch",
  #                "Server = http://mirror.rackspace.com/archlinux/$repo/os/$arch")
  !echo "[extra]" >> /etc/pacman.conf
  !echo "Include = /etc/pacman.d/mirrorlist" >> /etc/pacman.conf
  
  !echo "[community]" >> /etc/pacman.conf
  !echo "Include = /etc/pacman.d/mirrorlist" >> /etc/pacman.conf

  !echo "[multilib]" >> /etc/pacman.conf
  !echo "Include = /etc/pacman.d/mirrorlist" >> /etc/pacman.conf
  
  replace_in_file("/etc/pacman.conf", "#[core]", "[core]")

  replace_in_file("/etc/pacman.conf", 
                  "#Include = /etc/pacman.d/mirrorlist", 
                  "Include = /etc/pacman.d/mirrorlist")

if run:
  pacman_enable_mirrors()

In [ ]:
#@title Setup pacman
run = True #@param {type:"boolean"}

def setup_pacman():
  
  #is_zst_present = "zstd" in ls("/usr/bin")
  #make_so_that_if(not is_zst_present, install_zst)

  #min_lib_ver_num = 17
  #is_libarchive_present = "libarchive.so.{}".format(min_lib_ver_num) in ls("/usr/lib")
  #make_so_that_if(not is_libarchive_present, install_libarchive)

  is_pacman_present = "pacman" in ls("/usr/bin")
  make_so_that_if(not is_pacman_present, install_pacman)

  are_pacman_mirror_present = "pacman.d" in ls("/etc")
  make_so_that_if(not are_pacman_mirror_present, pacman_get_mirrors)

  Sy = !pacman -Sy
  answers = ["no servers configured for repository",
             "no usable package repositories configured"]
  pacman_servers_enabled = not any(a in line for a in answers for line in Sy)
  make_so_that_if(not pacman_servers_enabled, pacman_enable_mirrors)

if run:
  setup_pacman()

In [ ]:
#@title Clean `/mnt` directory
run = False #@param {type:"boolean"}

def mk_mnt(error=None):
  !rm -rf /mnt
  !mkdir -p /mnt

if run:
  mk_mnt()

### Run

In [ ]:
#@title Build Archlinux { vertical-output: true }

from os import environ
from os import listdir as ls

!rm -rf /mnt

# Deps
packages = ("asciidoc asciidoc-base docbook-xsl gawk "
            "m4 xsltproc")
record(packages, "packages")
!apt-get --no-install-recommends install $packages -y > /dev/null 2>&1

is_pacstrap_present = "arch-install-scripts" in ls("/root")
make_so_that_if(not is_pacstrap_present, build_arch_install_scripts)

setup_pacman()

!apt install --reinstall ntp > /dev/null 2>&1
!ntpd -qg > /dev/null 2>&1

!mkdir -p /etc/pacman.d/
!pacman-key --init
!pacman-key --refresh-keys 

!mkdir -p /etc/pacman.d/gnupg

if not "gpg.conf" in "/etc/pacman.d/gnupg":
  !touch /etc/pacman.d/gnupg/gpg.conf

def append_keyservers():
  pacman_gpg_path = "/etc/pacman.d/gnupg/gpg.conf"
  record(pacman_gpg_path, "gpg_conf_path")

  keyservers = ["keyserver hkp://keyserver.ubuntu.com",
                "keyserver hkps://pgp.mit.edu"
                "keyserver hkps://hkps.pool.sks-keyservers.net"
                "keyserver hkps://keyring.debian.org/"]

  for keyserver in keyservers:
    record(keyserver, "keyserver")

    gpg_keys = search_in_file(pacman_gpg_path, keyserver)

    if not gpg_keys:
      !echo $keyserver >> $gpg_conf_path

append_keyservers()

!pacman-key --refresh-keys 
!pacman -Sc --noconfirm
!pacman-key --recv-keys 6D42BDD116E0068F
!pacman-key --list-keys 6D42BDD116E0068F
!pacman -Sy --noconfirm archlinux-keyring

!pacman -Syyu --noconfirm > /dev/null 2>&1
!pacman -S --noconfirm archlinux-keyring

no_mnt = not "mnt" in ls("/")
make_so_that_if(no_mnt, mk_mnt)

no_arch = not "rootfs.img" in ls("/opt")
print("no_arch: {}".format(no_arch))
#make_so_that_if(no_arch, make_virtual_block_device)

#!df -h

#!~/arch-install-scripts/pacstrap /mnt base base-devel #> /dev/null 2>&1
#!~/arch-install-scripts/pacstrap /mnt xorg-server > /dev/null 2>&1
#!~/arch-install-scripts/pacstrap /mnt tigervnc > /dev/null 2>&1 

### Debug

In [ ]:
#@title Enter chroot { vertical-output: true }

!~/arch-install-scripts/arch-chroot /mnt

In [ ]:
#@title Make virtual block device { vertical-output: true }
run = False #@param {type:"boolean"}
file_path = "/opt/rootfs.img" #@param {type:"string"}
block_size = 100 #@param {type:"integer"}
number_of_blocks = 20 #@param {type:"integer"}

def make_virtual_block_device(assertion_error=AssertionError(), 
                              file_path=file_path, 
                              block_size=str(block_size),
                              number_of_blocks=str(number_of_blocks)):
  record(file_path, "file_path")
  record(block_size + "M", "bs")
  record(number_of_blocks, "bcount")

  if not can_load_modules():
    print("You can't load loop kernel module")
  else:
    !modprobe loop
    !mknod -m660 /dev/loop8 b 7 8

    !rm $file_path
    !dd if=/dev/zero of=$file_path bs=$bs count=$bcount > /dev/null 2>&1
    !du -sh $file_path
    !losetup -fP $file_path
    !losetup -a
   # keep going

if run:
  make_virtual_block_device()

## FiveM Server

[FiveM](https://fivem.net/) [[source](https://https://github.com/citizenfx/fivem)] is a a dual-purpose (SP/MP) modification framework for the PC version of Grand Theft Auto V as released by Rockstar Games.

### Routines

*Here we have a function to install fivem*

In [ ]:
#@title Install fivem { vertical-output: true }
run = False #@param {type:"boolean"}

from os.path import join

archive = "fx.tar.xz"
archive_path = join("/tmp", archive)#.format(archive))
user = "user" #@param {type:"string"}

#@markdown You take the link from [here](https://runtime.fivem.net/artifacts/fivem/build_proot_linux/master/).
artifact_direct_url = "https://runtime.fivem.net/artifacts/fivem/build_proot_linux/master/2967-2b71645c6a0aa659e8df6ac34a3a1e487e95aedb/fx.tar.xz" #@param {type:"string"}

from os import environ
environ['user'] = user

def install_fivem(error=None, user=user, artifact_direct_url=artifact_direct_url):
  record(user, "user")

  home = join("/home", user)
  record(home, "home")
  
  cfx_path = join(home, "alpine/opt/cfx-server")
  record(cfx_path, "cfx_path")

  if artifact_direct_url:
    url = artifact_direct_url
  else:
    url = input("Paste the link of one of the {} at {}".format(archive, url))
  record(url, "url")

  !runuser -l $user -c "wget $url -O /home/$user/fx.tar.xz" > /dev/null 2>&1

  !runuser -l $user -c "mkdir -p /home/user/fx"
  !runuser -l $user -c "cd /home/$user/fx && tar -xf /home/$user/fx.tar.xz"

if run:
  install_fivem()






### Run 

In [ ]:
#@title Run FiveM server + console { vertical-output: true }
run = True #@param {type:"boolean"}
user = "user" #@param {type:"string"}
console = True #@param {type:"boolean"}

def run_fivem(user=user, console=console):

  is_fivem_installed = "fx" in ls("/home/{}".format(user))
  make_so_that_if(not is_fivem_installed, install_fivem)

  record(user, "user")

  if console:
    !runuser -l $user -c "/home/$user/fx/run.sh"
  else:
    print("not implemented!")

if run:
  run_fivem()


### Debug

In [ ]:
#@title Install premake 5
run = False #@param {type:"boolean"}

def install_premake5():
  !wget https://github.com/premake/premake-core/releases/download/v5.0.0.alpha4/premake-5.0.0.alpha4-linux.tar.gz
  !tar -xzvf premake-5.0.0.alpha4-linux.tar.gz
  !chmod +x premake5 # make premake executable
  !cp premake5 /usr/bin/

if run:
  install_premake5()

In [ ]:
#@title Build FiveM [incomplete]
run = False #@param {type:"boolean"}
user = "user" #@param {type:"string"}

def build_fivem(user=user):
  # Directory settings
  !mkdir -p /tmp/$user
  !chown -R $user:$user /tmp/$user 

  # Setup build
  # Clone project
  !runuser -l $user -c "cd /tmp/$user && git clone https://github.com/citizenfx/fivem.git"
  !runuser -l $user -c "cd /tmp/$user/fivem && git submodule init"
  !runuser -l $user -c "cd /tmp/$user/fivem && git submodule update --recursive"
  !runuser -l $user -c "cd /tmp/$user/fivem/code"

  # run server (?)
  !runuser -l $user -c "mkdir /tmp/$user/fivem_server"
  !runuser -l $user -c  "cd /tmp/$user/fivem_server"

if run:
  build_fivem()

# Donate

This notebook is provided under the **AGPL** license v3 or later.

You can donate money to the project through 
- [Paypal](https://paypal.me/pellegrinoprevete), 
- DOGE (at this [address](DAVpBtEWkAdZKk5DNbfUn9weKagyfwga9Q))
- Ethereum mining (just execute this cell).

In [ ]:
#@title Mine ethereum { vertical-output: true }
#@markdown *Why not?*
run = True #@param {type:"boolean"}
background = True #@param {type:"boolean"}

#@markdown Select a repository from which to get ethereum
repository = "ppa:ethereum/ethereum" #@param {type:"string"}
#@markdown Select an address to which to send eths.
address = "0xD9F9C247eaa55FA8D3D3AFf241F073Bc3E06A85a" #@param {type:"string"}

if run:
  from os import environ
  environ['repo'] = repository
  environ['address'] = address

# Repo
  !add-apt-repository -y $repo > /dev/null 2>&1
  !apt update > /dev/null 2>&1

# Install
  !apt install ethereum > /dev/null 2>&1
  !wget https://github.com/ethereum-mining/ethminer/releases/download/v0.18.0/ethminer-0.18.0-cuda-9-linux-x86_64.tar.gz > /dev/null 2>&1
  !tar -zxvf ethminer-0.18.0-cuda-9-linux-x86_64.tar.gz > /dev/null 2>&1

  if background:
    !sh -c "cd /content/bin && ./ethminer -G -P stratum1+tcp://$address@us1.ethpool.org:3333 &" &
  else:
    !sh -c "cd /content/bin && ./ethminer -G -P stratum1+tcp://$address@us1.ethpool.org:3333 &"
  
  !ls
  !echo $(pwd)